### Dataframes, Spark SQL e Parquet - practica

In [1]:
# Carga o ficheiro purchases.txt desde o sistema de ficheiros localabs

In [17]:
df = spark.read.csv("file:///home/hduser/Descargas/purchases.txt", sep='\t')

df = df.toDF('fecha','hora','tenda','categoria','precio','tipo_pago')

In [20]:
# Mostra os datos do dataframe

In [21]:
df.show()

+----------+-----+--------------+--------------------+------+----------+
|     fecha| hora|         tenda|           categoria|precio| tipo_pago|
+----------+-----+--------------+--------------------+------+----------+
|2012-01-01|09:00|      San Jose|      Men's Clothing|214.05|      Amex|
|2012-01-01|09:00|    Fort Worth|    Women's Clothing|153.57|      Visa|
|2012-01-01|09:00|     San Diego|               Music| 66.08|      Cash|
|2012-01-01|09:00|    Pittsburgh|        Pet Supplies|493.51|  Discover|
|2012-01-01|09:00|         Omaha| Children's Clothing|235.63|MasterCard|
|2012-01-01|09:00|      Stockton|      Men's Clothing|247.18|MasterCard|
|2012-01-01|09:00|        Austin|             Cameras| 379.6|      Visa|
|2012-01-01|09:00|      New York|Consumer Electronics| 296.8|      Cash|
|2012-01-01|09:00|Corpus Christi|                Toys| 25.38|  Discover|
|2012-01-01|09:00|    Fort Worth|                Toys|213.88|      Visa|
|2012-01-01|09:00|     Las Vegas|         Video Gam

In [2]:
# Crea unha táboa sobre o dataframe para poder realizar consultas SQL

In [27]:
df.createOrReplaceTempView('vendas')

In [28]:
# Mostra os datos da táboa cunha consulta SQL

In [29]:
spark.sql('SELECT * FROM vendas').show()

+----------+-----+--------------+--------------------+------+----------+
|     fecha| hora|         tenda|           categoria|precio| tipo_pago|
+----------+-----+--------------+--------------------+------+----------+
|2012-01-01|09:00|      San Jose|      Men's Clothing|214.05|      Amex|
|2012-01-01|09:00|    Fort Worth|    Women's Clothing|153.57|      Visa|
|2012-01-01|09:00|     San Diego|               Music| 66.08|      Cash|
|2012-01-01|09:00|    Pittsburgh|        Pet Supplies|493.51|  Discover|
|2012-01-01|09:00|         Omaha| Children's Clothing|235.63|MasterCard|
|2012-01-01|09:00|      Stockton|      Men's Clothing|247.18|MasterCard|
|2012-01-01|09:00|        Austin|             Cameras| 379.6|      Visa|
|2012-01-01|09:00|      New York|Consumer Electronics| 296.8|      Cash|
|2012-01-01|09:00|Corpus Christi|                Toys| 25.38|  Discover|
|2012-01-01|09:00|    Fort Worth|                Toys|213.88|      Visa|
|2012-01-01|09:00|     Las Vegas|         Video Gam

In [5]:
# Conta o número de vendas por tenda

In [33]:
spark.sql("SELECT tenda, count(*) FROM vendas GROUP BY tenda").show()

+---------------+--------+
|          tenda|count(1)|
+---------------+--------+
|North Las Vegas|   40013|
|        Phoenix|   40333|
|          Omaha|   40209|
|      Anchorage|   39806|
|        Anaheim|   40086|
|     Greensboro|   40232|
|         Dallas|   40368|
|        Oakland|   39728|
|         Laredo|   40342|
|     Scottsdale|   40173|
|    San Antonio|   40197|
|    Bakersfield|   40326|
|        Raleigh|   40261|
|    Chula Vista|   40080|
|   Philadelphia|   40748|
|     Louisville|   40099|
|    Los Angeles|   40254|
|       Chandler|   39826|
|     Sacramento|   40561|
|   Indianapolis|   40321|
+---------------+--------+
only showing top 20 rows



In [6]:
# Fai un total de vendas por categoría

In [41]:
spark.sql("SELECT categoria, SUM(precio) FROM vendas GROUP BY categoria").show()



+--------------------+--------------------+
|           categoria|         sum(precio)|
+--------------------+--------------------+
| Children's Clothing| 5.762482093999994E7|
|      Sporting Goods| 5.759908588999996E7|
|                 CDs| 5.741075303999995E7|
|           Computers| 5.731540632000032E7|
|Consumer Electronics|5.7452374130000055E7|
|   Health and Beauty|5.7481589560001045E7|
|        Pet Supplies| 5.719725023999971E7|
|                DVDs| 5.764921214000037E7|
|                Baby|5.7491808440000996E7|
|              Crafts| 5.741815449999973E7|
|    Women's Clothing| 5.743444896999931E7|
|         Video Games| 5.751316558000001E7|
|               Books| 5.745075790999974E7|
|               Music| 5.749548970000029E7|
|      Men's Clothing| 5.762127904000029E7|
|             Cameras| 5.729904664000106E7|
|              Garden| 5.753983310999994E7|
|                Toys| 5.746347710999978E7|
+--------------------+--------------------+



In [7]:
# Garda o dataframe en HDFS en formato parquet, con particións para as diferentes tendas

In [44]:
df.write.partitionBy("tenda").mode("overwrite").parquet("output/exercicio_04/vendas_por_tenda.parquet")

In [8]:
# Carga os datos da tenda de San Jose

In [54]:
df_vendas_San_Jose = spark.read.parquet("output/exercicio_04/vendas_por_tenda.parquet/tenda=San Jose")


In [9]:
# Mostra os datos do dataframe

In [55]:
df_vendas_San_Jose.show()

+----------+-----+--------------------+------+----------+
|     fecha| hora|           categoria|precio| tipo_pago|
+----------+-----+--------------------+------+----------+
|2012-04-16|09:22|                 CDs| 94.52|  Discover|
|2012-01-01|09:00|      Men's Clothing|214.05|      Amex|
|2012-04-16|09:29|Consumer Electronics|381.52|MasterCard|
|2012-01-01|09:00|    Women's Clothing|215.82|      Cash|
|2012-04-16|09:36|      Sporting Goods|145.74|      Amex|
|2012-01-01|09:09|                Toys|337.71|      Cash|
|2012-04-16|09:37|                DVDs|212.34|      Amex|
|2012-01-01|09:17|              Garden|192.82|      Cash|
|2012-04-16|09:51| Children's Clothing|105.58|MasterCard|
|2012-01-01|09:19|             Cameras| 95.81|      Cash|
|2012-04-16|09:53|           Computers|307.14|      Cash|
|2012-01-01|09:19|        Pet Supplies|253.33|  Discover|
|2012-04-16|09:55|                Baby| 141.5|      Visa|
|2012-01-01|09:20|           Computers| 160.6|      Amex|
|2012-04-16|09

In [10]:
# Crea unha táboa sobre o dataframe dos datos de San Jose

In [56]:
df_vendas_San_Jose.createOrReplaceTempView('vista_vendas_San_Jose')

In [11]:
# Mostra os datos da táboa

In [57]:
spark.sql('SELECT * FROM vista_vendas_San_Jose').show()

+----------+-----+--------------------+------+----------+
|     fecha| hora|           categoria|precio| tipo_pago|
+----------+-----+--------------------+------+----------+
|2012-04-16|09:22|                 CDs| 94.52|  Discover|
|2012-01-01|09:00|      Men's Clothing|214.05|      Amex|
|2012-04-16|09:29|Consumer Electronics|381.52|MasterCard|
|2012-01-01|09:00|    Women's Clothing|215.82|      Cash|
|2012-04-16|09:36|      Sporting Goods|145.74|      Amex|
|2012-01-01|09:09|                Toys|337.71|      Cash|
|2012-04-16|09:37|                DVDs|212.34|      Amex|
|2012-01-01|09:17|              Garden|192.82|      Cash|
|2012-04-16|09:51| Children's Clothing|105.58|MasterCard|
|2012-01-01|09:19|             Cameras| 95.81|      Cash|
|2012-04-16|09:53|           Computers|307.14|      Cash|
|2012-01-01|09:19|        Pet Supplies|253.33|  Discover|
|2012-04-16|09:55|                Baby| 141.5|      Visa|
|2012-01-01|09:20|           Computers| 160.6|      Amex|
|2012-04-16|09

In [64]:
# Consulta o total de vendas de San José por tipo de elementos (categoría)

In [68]:
spark.sql("SELECT categoria, SUM(precio) FROM vista_vendas_San_Jose GROUP BY categoria").show()


+--------------------+------------------+
|           categoria|       sum(precio)|
+--------------------+------------------+
| Children's Clothing| 549454.2400000003|
|      Sporting Goods| 540678.4100000004|
|                 CDs| 586499.6699999999|
|           Computers| 561784.6999999997|
|Consumer Electronics| 556462.0700000002|
|   Health and Beauty| 553600.1900000004|
|        Pet Supplies| 540508.1199999999|
|                DVDs| 569417.1900000004|
|                Baby|         566853.52|
|              Crafts|         548947.01|
|    Women's Clothing| 540482.1599999997|
|         Video Games| 573047.9199999999|
|               Books|522989.79999999993|
|               Music| 550085.7599999993|
|      Men's Clothing| 551148.6200000005|
|              Garden| 559343.9099999995|
|             Cameras|         527568.34|
|                Toys|         537849.78|
+--------------------+------------------+

